<a href="https://colab.research.google.com/github/ipejun-ai/m5-accuracy/blob/master/point_to_uncertainty_different_ranges_per_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you like it, please upvote:)

My aim is to show that uncertainty, in some sense, decreases on higher levels of aggregation.

The bulk of the work in this kernel is done by [@kneroma](https://www.kaggle.com/kneroma) in [this kernel](https://www.kaggle.com/kneroma/from-point-to-uncertainty-prediction), so make sure to check it out.



In [0]:
import pandas as pd, numpy as np
from matplotlib import pyplot as plt

import scipy.stats  as stats

In [0]:
pd.options.display.max_columns = 50

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
DIRPATH="/content/gdrive/My Drive/kaggle/"
#DIRPATH="C:/Users/peiju/Documents/Study/kaggle/m5-forecasting-accuracy/"

In [0]:
VER=4

In [18]:
best = pd.read_csv(DIRPATH+'/output/m5-three-shades-of-dark-darker-magic/submission_v'+str(VER)+".csv")
best.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.827091,0.752164,0.745912,0.757811,0.951325,1.147378,1.039044,0.827880,0.890095,0.790337,0.749971,0.859571,0.981799,0.921299,0.820556,0.817720,0.757708,0.743817,0.799119,0.998937,0.927218,0.808440,0.780806,0.777631,0.829567,0.905376,1.024064,0.960905
1,HOBBIES_1_002_CA_1_validation,0.177803,0.166564,0.158272,0.173692,0.207080,0.262814,0.289829,0.229599,0.216330,0.204090,0.185191,0.208048,0.252560,0.267916,0.251112,0.227614,0.201146,0.191648,0.214474,0.267069,0.269160,0.220142,0.190172,0.201600,0.215206,0.212885,0.314655,0.314827
2,HOBBIES_1_003_CA_1_validation,0.447317,0.433442,0.473743,0.473401,0.642051,0.793468,0.662483,0.510270,0.470970,0.458860,0.437056,0.537279,0.692188,0.606010,0.449477,0.417642,0.411018,0.485203,0.601274,0.730618,0.707351,0.506976,0.460727,0.485214,0.486676,0.626753,0.718936,0.725013
3,HOBBIES_1_004_CA_1_validation,1.645193,1.313597,1.361617,1.528152,1.999726,3.139540,2.904124,1.655795,1.349791,1.371399,1.434756,1.686546,2.817873,3.022482,1.661411,1.490736,1.444336,1.361693,1.813723,2.377692,2.942369,1.715313,1.462247,1.350195,1.395161,1.970927,2.895628,3.176404
4,HOBBIES_1_005_CA_1_validation,0.846102,0.829958,0.820962,0.896291,1.135508,1.487340,1.475425,0.973355,0.968118,0.980621,0.905800,1.106419,1.398137,1.277246,0.952044,0.907224,0.916511,0.955970,1.133723,1.453961,1.431366,1.007006,0.955084,0.938126,0.933016,1.202956,1.465639,1.425902


In [19]:
sales = pd.read_csv(DIRPATH+"/m5-forecasting-uncertainty/sales_train_validation.csv")
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [20]:
sub = best.merge(sales[["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]], on = "id")
sub["_all_"] = "Total"
sub.shape

(30490, 35)

In [21]:
sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,item_id,dept_id,cat_id,store_id,state_id,_all_
0,HOBBIES_1_001_CA_1_validation,0.827091,0.752164,0.745912,0.757811,0.951325,1.147378,1.039044,0.827880,0.890095,0.790337,0.749971,0.859571,0.981799,0.921299,0.820556,0.817720,0.757708,0.743817,0.799119,0.998937,0.927218,0.808440,0.780806,0.777631,0.829567,0.905376,1.024064,0.960905,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,Total
1,HOBBIES_1_002_CA_1_validation,0.177803,0.166564,0.158272,0.173692,0.207080,0.262814,0.289829,0.229599,0.216330,0.204090,0.185191,0.208048,0.252560,0.267916,0.251112,0.227614,0.201146,0.191648,0.214474,0.267069,0.269160,0.220142,0.190172,0.201600,0.215206,0.212885,0.314655,0.314827,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,Total
2,HOBBIES_1_003_CA_1_validation,0.447317,0.433442,0.473743,0.473401,0.642051,0.793468,0.662483,0.510270,0.470970,0.458860,0.437056,0.537279,0.692188,0.606010,0.449477,0.417642,0.411018,0.485203,0.601274,0.730618,0.707351,0.506976,0.460727,0.485214,0.486676,0.626753,0.718936,0.725013,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,Total
3,HOBBIES_1_004_CA_1_validation,1.645193,1.313597,1.361617,1.528152,1.999726,3.139540,2.904124,1.655795,1.349791,1.371399,1.434756,1.686546,2.817873,3.022482,1.661411,1.490736,1.444336,1.361693,1.813723,2.377692,2.942369,1.715313,1.462247,1.350195,1.395161,1.970927,2.895628,3.176404,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,Total
4,HOBBIES_1_005_CA_1_validation,0.846102,0.829958,0.820962,0.896291,1.135508,1.487340,1.475425,0.973355,0.968118,0.980621,0.905800,1.106419,1.398137,1.277246,0.952044,0.907224,0.916511,0.955970,1.133723,1.453961,1.431366,1.007006,0.955084,0.938126,0.933016,1.202956,1.465639,1.425902,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,Total


## Different ratios for different aggregation levels

The higher the aggregation level, the more confident we are in the point prediction --> lower coef, relatively smaller range of quantiles

In [0]:
qs = np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995])
qs.shape


def get_ratios(coef=0.15):
    qs2 = np.log(qs/(1-qs))*coef
    ratios = stats.norm.cdf(qs2)
    ratios /= ratios[4]
    ratios = pd.Series(ratios, index=qs)
    return ratios.round(3)

#coef between 0.05 and 0.24 is used, probably suboptimal values for now

level_coef_dict = {"id": get_ratios(coef=0.3), "item_id": get_ratios(coef=0.15),
                   "dept_id": get_ratios(coef=0.08), "cat_id": get_ratios(coef=0.07),
                   "store_id": get_ratios(coef=0.08), "state_id": get_ratios(coef=0.07), "_all_": get_ratios(coef=0.05),
                   ("state_id", "item_id"): get_ratios(coef=0.19),  ("state_id", "dept_id"): get_ratios(coef=0.1),
                    ("store_id","dept_id") : get_ratios(coef=0.11), ("state_id", "cat_id"): get_ratios(coef=0.08),
                    ("store_id","cat_id"): get_ratios(coef=0.1)
                  }



Let's see how ranges differ!

In [23]:
level_coef_dict["id"]

0.005    0.112
0.025    0.272
0.165    0.627
0.250    0.742
0.500    1.000
0.750    1.258
0.835    1.373
0.975    1.728
0.995    1.888
dtype: float64

In [24]:
level_coef_dict["cat_id"]

0.005    0.711
0.025    0.798
0.165    0.910
0.250    0.939
0.500    1.000
0.750    1.061
0.835    1.090
0.975    1.202
0.995    1.289
dtype: float64

For the the lowest level (30490 series), the smallest and biggest quantiles are 20% and 180% of the point prediction. For categories (3 series), the model will be way more confident: the smallest quantile will be 71%, the biggest will be 129% of the point prediction.

In [0]:


def quantile_coefs(q, level):
    ratios = level_coef_dict[level]
               
    return ratios.loc[q].values

def get_group_preds(pred, level):
    df = pred.groupby(level)[cols].sum()
    q = np.repeat(qs, len(df))
    df = pd.concat([df]*9, axis=0, sort=False)
    df.reset_index(inplace = True)
    df[cols] *= quantile_coefs(q, level)[:, None]
    if level != "id":
        df["id"] = [f"{lev}_X_{q:.3f}_validation" for lev, q in zip(df[level].values, q)]
    else:
        df["id"] = [f"{lev.replace('_validation', '')}_{q:.3f}_validation" for lev, q in zip(df[level].values, q)]
    df = df[["id"]+list(cols)]
    return df

def get_couple_group_preds(pred, level1, level2):
    df = pred.groupby([level1, level2])[cols].sum()
    q = np.repeat(qs, len(df))
    df = pd.concat([df]*9, axis=0, sort=False)
    df.reset_index(inplace = True)
    df[cols] *= quantile_coefs(q, (level1, level2))[:, None]
    df["id"] = [f"{lev1}_{lev2}_{q:.3f}_validation" for lev1,lev2, q in 
                zip(df[level1].values,df[level2].values, q)]
    df = df[["id"]+list(cols)]
    return df

levels = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "_all_"]
couples = [("state_id", "item_id"),  ("state_id", "dept_id"),("store_id","dept_id"),
                            ("state_id", "cat_id"),("store_id","cat_id")]
cols = [f"F{i}" for i in range(1, 29)]

df = []
for level in levels :
    df.append(get_group_preds(sub, level))
for level1,level2 in couples:
    df.append(get_couple_group_preds(sub, level1, level2))
df = pd.concat(df, axis=0, sort=False)
df.reset_index(drop=True, inplace=True)
df = pd.concat([df,df] , axis=0, sort=False)
df.reset_index(drop=True, inplace=True)
df.loc[df.index >= len(df.index)//2, "id"] = df.loc[df.index >= len(df.index)//2, "id"].str.replace(
                                    "_validation$", "_evaluation")

df.shape

df.head()

df.to_csv(DIRPATH+"/m5-forecasting-uncertainty/point-to-uncertainty-different-ranges-per-level/submissionv"+str(VER)+".csv", index = False)